In [10]:
import serial
import numpy as np
import matplotlib.pyplot as plt
import os.path
import pickle # para leer los archivos comprimidos (.p)

###parameters######################
COM_ARDUINO_UNO = 'COM7'
baudrate_arduino= 250000#115200 #250000 #
SampleFreq=2000 #Hz
tiempo_adq = 20 #seg
nombreArchivo = './datos/de201023_1.p'# nombre del archivo donde guardar los datos
nsensores =2 # 1 a 4
arduino_serial_plotter_protocol_on = 0 # data sent using arduino serial protocol (ascii for each digit) or send raw bytes

################################################

cantidad_samples = SampleFreq*tiempo_adq   #1min 120 000

def leer_sensor(arduino_puerto_serie,cantidad_samples):
    samples_sensor0 = np.zeros((cantidad_samples))
    #samples_sensor1 = np.zeros((cantidad_samples))
    #samples_sensor2 = np.zeros((cantidad_samples))
    if nsensores >= 2 :
        samples_sensor1 = np.zeros((cantidad_samples))
    if nsensores >= 3 :
        samples_sensor2 = np.zeros((cantidad_samples))                         
    if nsensores >= 4 :
        samples_sensor3 = np.zeros((cantidad_samples)) 
    i = 0
    arduino.reset_input_buffer()
    for n_samples in range(0,cantidad_samples):
        read_val = False
        splitted = ""
        while not read_val:
            try:
                if arduino_serial_plotter_protocol_on == 0:
                    a = arduino.read(nsensores*3) #nbytes= 3bytes x sensor( 2bytes datos+1 de coma o de /n el ultimo)
                    if len(a) == nsensores*3 :
                        samples_sensor0[n_samples] = float(a[1] + 256*a[0])
                        if nsensores >= 2 :
                            samples_sensor1[n_samples] = float(a[4] + 256*a[3]) 
                        if nsensores >= 3 :
                            samples_sensor2[n_samples] = float(a[7] + 256*a[6])                         
                        if nsensores >= 4 :
                            samples_sensor3[n_samples] = float(a[10] + 256*a[9])                         
                        #samples_sensor1[n_samples] = float(a[4] + 256*a[3])
                        #samples_sensor2[n_samples] = float(a[7] + 256*a[6])
                        read_val = True
                    else:
                        print("SYNCHRONIZING COMMUNICATION: serial read didnt receive amount of bytes as expected. data discarded")
                    a = []
                if arduino_serial_plotter_protocol_on == 1: #adquiero con formato arduino serial plotter
                    line = arduino.readline()
                    string_n = line.decode()  # decode byte string into Unicode  
                    string = string_n.rstrip() # remove \n and \r
                    string_splitted = string.split(",")
                    samples_sensor0[n_samples] = float(string_splitted[0])
                    samples_sensor1[n_samples] = float(string_splitted[1])
                    read_val = True
                #print(i)
                #i+=1
            except ValueError:
                print("...")
                #print("Línea con error: {}".format(line))
# creo un diccionario para guardar mis datos
    if nsensores == 1 :
        samples = {'sensor0' : samples_sensor0} 
    if nsensores == 2 :
        samples = {'sensor0' : samples_sensor0,'sensor1' : samples_sensor1} 
    if nsensores == 3 :
        samples = {'sensor0' : samples_sensor0,'sensor1' : samples_sensor1, 'sensor2' : samples_sensor2} 
    if nsensores == 4 :
        samples = {'sensor0' : samples_sensor0,'sensor1' : samples_sensor1, 'sensor2' : samples_sensor2, 'sensor3' : samples_sensor3} 

    return samples #samples_sensor0, samples_sensor1, samples_sensor2


def guardar_datos(nombreArchivo, datos):
    # comprueba si el archivo existe
    if os.path.exists(nombreArchivo):
        print("El archivo %s ya existe. Renombrando..." %nombreArchivo)
        # chequear version
        v = 0
        path = os.path.split(nombreArchivo)[0]
        raw = os.path.splitext(os.path.split(nombreArchivo)[1])[0]
        ext = os.path.splitext(os.path.split(nombreArchivo)[1])[1]
        if len(raw.split('_')) < 2:
            v += 1
            new = raw + '_'+str(v)+ext
        else:
            v = int(raw.split('_')[1]) + 1
            new = raw.split('_')[0]+'_'+str(v)+ext
        nombreArchivo = os.path.join(path, new)
    with open(nombreArchivo, 'wb') as f:
        pickle.dump(datos, f)
        print("Datos guardados en %s" %nombreArchivo)
        f.close()
    return nombreArchivo

In [11]:
# Abrimos la conexión con Arduino, adquiero, y guardo
# Hack cortesía de https://www.oreilly.com/library/view/python-cookbook/0596001673/ch17s02.html
try: arduino
except NameError:
    arduino = serial.Serial(COM_ARDUINO_UNO, baudrate=baudrate_arduino, timeout=1.0)
if not arduino.is_open: arduino.open()
arduino.setDTR(False)
arduino.flushInput()
arduino.setDTR(True)
print('adquiriendo...')
samples = leer_sensor(arduino,cantidad_samples)
arduino.close()

samples_sensor0 = samples['sensor0']
if nsensores >= 2 :
    samples_sensor1 = samples['sensor1']
if nsensores >= 3 :
    samples_sensor2 = samples['sensor2']
if nsensores >= 4 :
    samples_sensor3 = samples['sensor3']
nombreArchivo = guardar_datos(nombreArchivo, samples)

# grafico de muestras completas
time = np.arange(0,cantidad_samples/SampleFreq,1/SampleFreq)
fig = plt.figure()
plt.plot(time,samples_sensor0, 'b', label="Sensor A")
if nsensores >= 2 :
    plt.plot(time,samples_sensor1, 'r', label="Sensor B")
if nsensores >= 3 :
    plt.plot(time,samples_sensor2, 'k', label="Sensor 3")
if nsensores >= 4 :
    plt.plot(time,samples_sensor3, 'g', label="Sensor 4")

plt.xlabel("Tiempo [ms]")
plt.ylabel("Señal (cuentas)")
plt.title("PULSOS ADQUIRIDOS")
plt.legend()
#plt.xlim([1000,30000])
plt.grid()
plt.show()


adquiriendo...
SYNCHRONIZING COMMUNICATION: serial read didnt receive amount of bytes as expected. data discarded


FileNotFoundError: [Errno 2] No such file or directory: './datos/de201023_1.p'